In [1]:
import enchant
from PIL import Image, ImageEnhance
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
import cv2
import re 
import matplotlib.pyplot as plt
import numpy as np
from imgProcessor.measure.sharpness import parameters
from skimage.util import random_noise
# from skimage.exposure import is_low_contrast
# import skimage.measure
import imutils
from skimage.transform import rotate
from scipy.signal import convolve2d
import math
from skimage.restoration import estimate_sigma


In [2]:
def estimateNoise(img):
    return estimate_sigma(img, multichannel=True, average_sigmas=True)

def accuracyByLevenshteinDistance(textOCR,label):
    #tira espaços, tab e \n
    text = re.sub(r"[\n\t\s]*", "", textOCR)
    labelText = re.sub(r"[\n\t\s]*", "", label)
    
    #calcula a distancia de levenshtein entre os textos
    levDistance = enchant.utils.levenshtein(text, labelText)
    
    #calcula a acurácia fazendo accuracy = n-erros/n
    accuracy = (len(labelText)-levDistance) / len(labelText)
    return accuracy

def blurLevel(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

def instantiate_histogram():    
    histArray= []
    
    for i in range(0,256):
        histArray.append(str(i))
        histArray.append(0)
    
    hist = {histArray[i]: histArray[i + 1] for i in range(0, len(histArray), 2)} 
    
    return hist

def count_intensity_values(hist, img):
    for row in range(img.shape[0]):
        for column in range(img.shape[1]):
            hist[str(img[row][column])] += 1
     
    return hist

def plotHist(hist, hist2=''):
    if hist2 != '':
        figure, axarr = plt.subplots(1,2, figsize=(20, 10))
        axarr[0].bar(hist.keys(), hist.values())
        axarr[1].bar(hist2.keys(), hist2.values())
    else:
        plt.bar(hist.keys(), hist.values())
        plt.xlabel("Níveis intensidade")
        ax = plt.gca()
        ax.axes.xaxis.set_ticks([])
        plt.grid(True)
        plt.show()
        
def get_hist_proba(hist, n_pixels):
    hist_proba = {}
    for i in range(0, 256):
        hist_proba[str(i)] = hist[str(i)] / n_pixels
    
    return hist_proba

def getMediumBrightness(img):
    hist = instantiate_histogram()
    hist = count_intensity_values(hist, img)
    n = img.shape[0] * img.shape[1]
    histP = get_hist_proba(hist, n)
    
    L = img.max()
    mediumBright = 0
    
    for i in range(256):
        mediumBright += i*histP[str(i)]
        
    return mediumBright

def biggest_contour(contours):
        biggest = np.array([])
        max_area = 0
        for i in contours:
            area = cv2.contourArea(i)
            if area > 58000:
                peri = cv2.arcLength(i, True)
                approx = cv2.approxPolyDP(i, 0.015 * peri, True)
                if area > max_area and len(approx) == 4:
                    biggest = approx
                    max_area = area
        return biggest
 

def order_points(pts):
        rect = np.zeros((4, 2), dtype = "float32")

        s = np.sum(pts,axis = 1)

        rect[0] = pts[np.argmin(s)]
        rect[2] = pts[np.argmax(s)]
        
        diff = np.diff(pts, axis = 1)
        rect[3] = pts[np.argmin(diff)]
        rect[1] = pts[np.argmax(diff)]
     
        # return the ordered coordinates
        return rect
       
def getAngle(img):
    image = imutils.resize(img, height = 500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    gray = cv2.bilateralFilter(gray, 20, 30, 30)
    edged = cv2.Canny(gray, 50, 150)
    
  
    cnts = None
    cnts, hierarchy  = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:10]

    
    screenCnt = None
    screenCnt = biggest_contour(cnts)
    
    if(len(screenCnt)==0 ):
        edged = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,13,0.5)
        cnts = None
        cnts, hierarchy  = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:10]
        
        screenCnt = None
        screenCnt = biggest_contour(cnts)

    if(len(screenCnt) >0 ):
        cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 1)
        screenCnt = order_points(screenCnt[:,0,:])
        angle = cv2.minAreaRect(screenCnt)[-1]
        angle = 90 - angle if (angle>45) else angle         
        
        return angle
    else:
        return None


def estimate_noise(I):
    
    I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    Image  = cv2.Sobel(src=I, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5) #
    
    H, W = Image.shape

    M = [[1, -2, 1],
        [-2, 4, -2],
        [1, -2, 1]]

    sigma = np.sum(np.sum(np.absolute(convolve2d(Image, M))))

    sigma = sigma * (math.sqrt(math.pi/2) * (1 /  (6 * (W-2) * (H-2))))

    return sigma

In [3]:
def newEst(I):
    I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
   
    M = [[1, -2, 1],
        [-2, 4, -2],
        [1, -2, 1]]
    
    im = convolve2d(I, M)
    Image  = cv2.Sobel(src=im, ddepth=cv2.CV_32SC1, dx=1, dy=1, ksize=3)
    
    H, W = Image.shape
    sigma = np.sum(np.sum(np.absolute(Image)))

    sigma = sigma * (math.sqrt(math.pi/2) * (1 /  (6 * (W-2) * (H-2))))

    return sigma

In [4]:
import random
import numpy as np

# Criar o array 3 x 3 com números aleatórios entre 1 e 52
np.random.seed(2)
x = np.random.randint(0,200, (40))
print(x)

[168  15  72  22  43  75 104   7 162 177  95  75  47  63  31 148 124 116
  37 167 195 102   4 170 107  51 103  38  33  58 124  67  69  88 196  46
 198  95 121  31]


In [23]:
acc = np.array([])
accOriginal = np.array([])
noiseLevelMore90 = np.array([])
noiseLevelLess15 = np.array([])
#blurLevelMore90noise = np.array([])
#blurLevelLess15noise = np.array([])
blurLevelMore90 = np.array([])
blurLevelLess15 = np.array([])
rotMore90 = np.array([])
rotLess15 = np.array([])


testenoise = np.array([])
testeblur = np.array([])
for i in x: # alterar testes para 20% do pipeline
    # print("---IMAGE 1"+str(i).zfill(3)+"---")
    img = np.array([])
    img = cv2.imread('./images/1'+str(i).zfill(3) +'-receipt.jpg')
    
    if (i == 25 or i == 34 or i == 41 or i == 45 or i == 63 or i == 109 or i == 137 or i == 169 or i == 194 or i == 192): continue
    # name = '1' + str(i).zfill(3)
    # print(accuracyAfter['Image'].to_list())
    # if(int(name) not in accuracyAfter['Image'].to_list()): continue
    
    
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
       
    # aplica efeito de blur com opencv
    blur_img = cv2.GaussianBlur(img,(5,5),3)
    
    # aplica ruido gaussiano
    noise_img = random_noise(img, mode='s&p')
    noise_img = (255*noise_img).astype(np.uint8)

    #rotaciona
    rotated = rotate(img, 15, resize=True) * 255
    rotated = (rotated).astype(np.uint8)
    
    #le a label salva
    label = open('./images/1'+str(i).zfill(3)+'-receipt.txt', 'r') 
    labelText = label.read()
     
    #pra cad aimg gera a pertubaçao mede a acuracia, e verifica se esta > 90% ou abaixo de 15% e guarda pra cada caso
    
    imgBlurLevel = blurLevel(img)    
    noiseLevel = estimate_noise(img)
    imgAngle = getAngle(img)
    
    text = pytesseract.image_to_string(img)
    accuracy = accuracyByLevenshteinDistance(text,labelText)
    #acc = np.append(acc,accuracyOriginal)
    acc = np.append(acc,accuracy)
    accOriginal = np.append(accOriginal,accuracy)
    
    testenoise = np.append(testenoise,noiseLevel)
    testeblur = np.append(testeblur,imgBlurLevel)
    
    if(accuracy > 0.9):
        noiseLevelMore90 = np.append(noiseLevelMore90, noiseLevel)
        blurLevelMore90 = np.append(blurLevelMore90,imgBlurLevel)
        if(imgAngle != None): 
            rotMore90 = np.append(rotMore90, imgAngle)
    elif(accuracy < 0.15):
        noiseLevelLess15 = np.append(noiseLevelLess15, noiseLevel)
        blurLevelLess15 = np.append(blurLevelLess15,imgBlurLevel)
        if(imgAngle != None): 
            rotLess15 = np.append(rotLess15, imgAngle)
        
    #img com blur
    text = pytesseract.image_to_string(blur_img)
    accuracy = accuracyByLevenshteinDistance(text,labelText)
    
    imgBlurLevel = blurLevel(blur_img)    
    noiseLevel = estimate_noise(blur_img)
    imgAngle = getAngle(blur_img)
    
    acc = np.append(acc,accuracy)
    testenoise = np.append(testenoise,noiseLevel)
    testeblur = np.append(testeblur,imgBlurLevel)
    
    if(accuracy > 0.9):
        noiseLevelMore90 = np.append(noiseLevelMore90, noiseLevel)
        blurLevelMore90 = np.append(blurLevelMore90,imgBlurLevel)   
        if(imgAngle != None): 
            rotMore90 = np.append(rotMore90, imgAngle)
    elif(accuracy < 0.15):
        noiseLevelLess15 = np.append(noiseLevelLess15, noiseLevel)
        blurLevelLess15 = np.append(blurLevelLess15,imgBlurLevel)   
        if(imgAngle != None): 
            rotLess15 = np.append(rotLess15, imgAngle)
        
    #img com ruido
    text = pytesseract.image_to_string(noise_img)
    accuracy = accuracyByLevenshteinDistance(text,labelText)
    
    imgBlurLevel = blurLevel(noise_img)    
    noiseLevel = estimate_noise(noise_img)
    imgAngle = getAngle(noise_img)
    
    acc = np.append(acc,accuracy)
    testenoise = np.append(testenoise,noiseLevel)
    testeblur = np.append(testeblur,imgBlurLevel)
        
    if(accuracy > 0.9):
        noiseLevelMore90 = np.append(noiseLevelMore90, noiseLevel)
        blurLevelMore90 = np.append(blurLevelMore90,imgBlurLevel)    
        if(imgAngle != None): 
            rotMore90 = np.append(rotMore90, imgAngle)
    elif(accuracy < 0.15):
        noiseLevelLess15 = np.append(noiseLevelLess15, noiseLevel)
        blurLevelLess15 = np.append(blurLevelLess15,imgBlurLevel)
        if(imgAngle != None): 
            rotLess15 = np.append(rotLess15, imgAngle)
    
    
    #img com rotacao
    text = pytesseract.image_to_string(rotated)
    accuracy = accuracyByLevenshteinDistance(text,labelText)
    
    imgBlurLevel = blurLevel(rotated)    
    noiseLevel = estimate_noise(rotated)
    imgAngle = getAngle(rotated)
    
    acc = np.append(acc,accuracy)
    testenoise = np.append(testenoise,noiseLevel)
    testeblur = np.append(testeblur,imgBlurLevel)
    if(accuracy > 0.9):
        noiseLevelMore90 = np.append(noiseLevelMore90, noiseLevel)
        blurLevelMore90 = np.append(blurLevelMore90,imgBlurLevel)
        if(imgAngle != None): 
            rotMore90 = np.append(rotMore90, imgAngle)
    elif(accuracy < 0.15):
        noiseLevelLess15 = np.append(noiseLevelLess15, noiseLevel)
        blurLevelLess15 = np.append(blurLevelLess15,imgBlurLevel)
        if(imgAngle != None): 
            rotLess15 = np.append(rotLess15, imgAngle)
    
    
    


63


In [6]:

print("acima 90")
print("noise")
print(np.mean(noiseLevelMore90))
print("blur")
print(np.mean(blurLevelMore90))
print("rotation")
print(np.mean(rotMore90))

print("abaixo 15")
print("noise")
print(np.mean(noiseLevelLess15))
print("blur")
print(np.mean(blurLevelLess15))
print("rotation")
print(np.mean(rotLess15))

acima 90
noise
13.40397381940906
blur
844.0292472089371
rotation
1.249346097310384
abaixo 15
noise
18.48451607148569
blur
4861.8834730371045
rotation
7.766774000470028


In [7]:
print("acima 90")
print("noise")
print(np.median(noiseLevelMore90))
print("blur")
print(np.median(blurLevelMore90))
print("rotation")
print(np.median(rotMore90))

print("abaixo 15")
print("noise")
print(np.median(noiseLevelLess15))
print("blur")
print(np.median(blurLevelLess15))
print("rotation")
print(np.median(rotLess15))

acima 90
noise
14.442693200682697
blur
1013.7069433681175
rotation
1.4528427124023438
abaixo 15
noise
6.384733263214752
blur
282.2634976393383
rotation
1.8806782960891724


In [8]:
# print("acima 90")
# print("noise")
# print(noiseLevelMore90)
# print("blur")
# print(blurLevelMore90)
# print("rotation")
# print(rotMore90)

# print("abaixo 15")
# print("noise")
# print(noiseLevelLess15)
# print("blur")
# print(blurLevelLess15)
# print("rotation")
# print(rotLess15)

In [9]:
print("acima 90")
print("noise")
print(np.max(noiseLevelMore90))
print(np.min(noiseLevelMore90))
print("blur")
print(np.max(blurLevelMore90))
print(np.min(blurLevelMore90))
print("rotation")
print(np.max(rotMore90))
print(np.min(rotMore90))

print("abaixo 15")
print("noise")
print(np.max(noiseLevelLess15))
print(np.min(noiseLevelLess15))
print("blur")
print(np.max(blurLevelLess15))
print(np.min(blurLevelLess15))
print("rotation")
print(np.max(rotLess15))
print(np.min(rotLess15))

acima 90
noise
15.310099020499983
8.811583354795228
blur
1090.232644098765
354.64849947568285
rotation
2.2951955795288086
0.0
abaixo 15
noise
72.34763517683794
1.0959089105501292
blur
27217.11566533095
4.2847189755004464
rotation
18.939002990722656
0.0


In [24]:
accuracyAll = np.array([])
accuracyGood = np.array([])
accuracyMedium = np.array([])
accuracyBad = np.array([])
accuracyAllOriginal = np.array([])
indexGood = np.array([])
indexMedium = np.array([])
indexBad = np.array([])



for i in x:
    img = np.array([])
    img = cv2.imread('./images/1'+str(i).zfill(3) +'-receipt.jpg')
    if (i == 25 or i == 34 or i == 41 or i == 45 or i == 63 or i == 109 or i == 137 or i == 169 or i == 194 or i == 192): continue
    
    
    # # aplica efeito de blur com opencv
    # blur_img = cv2.GaussianBlur(img,(5,5),3)
    
    # # aplica ruido gaussiano
    # noise_img = random_noise(img, mode='s&p')
    # noise_img = (255*noise_img).astype(np.uint8)

    # #rotaciona
    # rotated = rotate(img, 15, resize=True) * 255
    # rotated = (rotated).astype(np.uint8)
    
    
    label = open('./images/1'+str(i).zfill(3)+'-receipt.txt', 'r') 
    labelText = label.read()
    
    imgBlurLevel = blurLevel(img)
    noiseLevel = estimate_noise(img)
    imgAngle = getAngle(img)
    
    text =''
    text = pytesseract.image_to_string(img)
    accuracy = accuracyByLevenshteinDistance(text,labelText)
    accuracyAll = np.append(accuracyAll, accuracy)
    accuracyAllOriginal = np.append(accuracyAllOriginal,accuracy)
    
    #boas - aceitaveis
    if((imgBlurLevel <= 1090.24 and imgBlurLevel >= 354.63 ) and (noiseLevel>= 8.81 and noiseLevel <= 15.32 )): #verifica ruido e nitidez
        if(imgAngle != None): # verifica se conseguiu calcular um angulo
            if(imgAngle <= 2.30):# se encaixa em todos os parametros classifica como bom
                indexGood = np.append(indexGood,i)
                accuracyGood = np.append(accuracyGood, accuracy)
        else:
            indexGood = np.append(indexGood,i)
            accuracyGood = np.append(accuracyGood, accuracy)
            


    
    
for i in x:
    
    img = np.array([])
    img = cv2.imread('./images/1'+str(i).zfill(3) +'-receipt.jpg')
    if (i == 25 or i == 34 or i == 41 or i == 45 or i == 63 or i == 109 or i == 137 or i == 169 or i == 194 or i == 192): continue
    if(len(indexGood[indexGood == i] ) != 0): continue
    
    # # aplica efeito de blur com opencv
    # blur_img = cv2.GaussianBlur(img,(5,5),3)
    
    # # aplica ruido gaussiano
    # noise_img = random_noise(img, mode='s&p')
    # noise_img = (255*noise_img).astype(np.uint8)

    # #rotaciona
    # rotated = rotate(img, 15, resize=True) * 255
    # rotated = (rotated).astype(np.uint8)
    
    label = open('./images/1'+str(i).zfill(3)+'-receipt.txt', 'r') 
    labelText = label.read()
    
    imgBlurLevel = blurLevel(img)
    noiseLevel = estimate_noise(img)
    imgAngle = getAngle(img)
    
    text =''
    text = pytesseract.image_to_string(img)
    accuracy = accuracyByLevenshteinDistance(text,labelText)
    accuracyAll = np.append(accuracyAll, accuracy)
    
    
    #descartadas naive
    if((noiseLevel >= 19 and imgBlurLevel >= 5253) or imgBlurLevel <= 302.16):
        if(imgAngle != None):
            if(imgAngle > 7):
                indexBad = np.append(indexBad,i)
                accuracyBad = np.append(accuracyBad, accuracy)
            else:
                indexMedium = np.append(indexMedium,i)
                accuracyMedium = np.append(accuracyMedium, accuracy)
                
        else:
            indexBad = np.append(indexBad,i)
            accuracyBad = np.append(accuracyBad, accuracy)  
    else:
        indexMedium = np.append(indexMedium,i)
        accuracyMedium = np.append(accuracyMedium, accuracy)
        
  

0.9050847457627119
0.7039106145251397
0.8791208791208791
0.03861003861003861
0.3235294117647059
0.34615384615384615
0.7228070175438597
0.9002932551319648
0.7005988023952096
0.945
0.6190476190476191
0.34615384615384615
0.0
0.7670454545454546
0.48104956268221577
0.8838526912181303
0.8745247148288974
0.7298136645962733
0.10857142857142857
0.7264957264957265
0.9809885931558935
0.821256038647343
0.52465483234714
0.9530201342281879
0.7836734693877551
0.804416403785489
0.8858131487889274
0.684931506849315
0.0
0.8838526912181303
0.7676056338028169
0.6927536231884058
0.24343675417661098
0.6845878136200717
0.7246963562753036
0.7829268292682927
0.6190476190476191
0.21912350597609562
0.7670454545454546
0.7039106145251397
0.8791208791208791
0.3235294117647059
0.34615384615384615
0.7228070175438597
0.34615384615384615
0.0
0.48104956268221577
0.8838526912181303
0.10857142857142857
0.7264957264957265
0.8858131487889274
0.0
0.8838526912181303
0.6927536231884058
0.24343675417661098
0.6845878136200717
0.

In [22]:
# acuraria media das classificaçoes
print(len(accuracyAll))
print(np.mean(accuracyGood))
print(np.mean(accuracyMedium)) # esse array seriam as imagens a serem melhoradas
print(np.mean(accuracyBad)) 
print(len(accuracyAllOriginal))
print(len(accOriginal))

59
0.7466662095113253
0.5910464303745342
0.0
59
39
